In [1]:
import pandas as pd
from datetime import datetime
import requests
import json
import warnings

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    # path = '~/Documents/GitHub/f1-analytics/'
    path = '/Users/oliverjcarter/Documents/GitHub/f1-analytics/'

warnings.filterwarnings("ignore", category=RuntimeWarning) 
pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [2]:
races = pd.read_csv(path+'data/races.csv')

rounds = races[races.season == 2021]['round'].unique()

In [3]:
qual = {
    'season': [],
    'round': [],
    'position': [],
    'circuit_id':[],
    'driver': [],
    'q1': [],
    'q2': [],
    'q3': []
}

In [4]:
# query results API

for i in rounds:
    url = 'http://ergast.com/api/f1/{}/{}/qualifying.json'
    r = requests.get(url.format(2021, i))
    json = r.json()

    for item in json['MRData']['RaceTable']['Races']:
        for n in item['QualifyingResults']:
            qual['season'].append(2021)
            qual['round'].append(i)
            qual['position'].append(n['position'])
            qual['driver'].append(n['Driver']['driverId'])
            qual['circuit_id'].append(item['Circuit']['circuitId'])

            try:
                qual['q3'].append(n['Q3'])
            except:
                qual['q3'].append('OUT')

            try:
                qual['q2'].append(n['Q2'])
            except:
                qual['q2'].append('OUT')

            try:
                qual['q1'].append(n['Q1'])
            except:
                qual['q1'].append('DNF')

qualifying = pd.DataFrame(qual)

In [7]:
qualifying.head(20)

,season,round,position,circuit_id,driver,q1,q2,q3
0,2021,1,1,bahrain,max_verstappen,1:30.499,1:30.318,1:28.997
1,2021,1,2,bahrain,hamilton,1:30.617,1:30.085,1:29.385
2,2021,1,3,bahrain,bottas,1:31.200,1:30.186,1:29.586
3,2021,1,4,bahrain,leclerc,1:30.691,1:30.010,1:29.678
4,2021,1,5,bahrain,gasly,1:30.848,1:30.513,1:29.809
5,2021,1,6,bahrain,ricciardo,1:30.795,1:30.222,1:29.927
6,2021,1,7,bahrain,norris,1:30.902,1:30.099,1:29.974
7,2021,1,8,bahrain,sainz,1:31.653,1:30.009,1:30.215
8,2021,1,9,bahrain,alonso,1:30.863,1:30.595,1:30.249
9,2021,1,10,bahrain,stroll,1:31.261,1:30.624,1:30.601
